In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os


In [41]:
# --- Hyperparameter ---
batch_size = 32
img_size = (224, 224)
epochs = 20
dataset_dir = "dataset"

In [42]:
# --- Augmentasi hanya untuk training ---
train_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

val_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255)
])

In [43]:
# --- Load Dataset ---
train_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "train"),
    label_mode='binary',
    image_size=img_size,
    batch_size=batch_size
).map(lambda x, y: (train_augment(x), y))

val_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "val"),
    label_mode='binary',
    image_size=img_size,
    batch_size=batch_size
).map(lambda x, y: (val_augment(x), y))

Found 381 files belonging to 2 classes.
Found 381 files belonging to 2 classes.


In [44]:
# --- Prefetch untuk performa ---
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [45]:
# --- Load Pretrained Model (MobileNetV2) ---
base_model = MobileNetV2(input_shape=img_size + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze pretrained weights

In [46]:
# --- Tambah classifier baru ---
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


In [47]:
# --- Compile model ---
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [48]:
# --- Callbacks ---
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("truck_classifier_best_model.h5", save_best_only=True)
]

In [49]:
# --- Train model ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 728ms/step - accuracy: 0.6087 - loss: 0.6735

12/12 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.6125 - loss: 0.6688 - val_accuracy: 0.8766 - val_loss: 0.3876
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 738ms/step - accuracy: 0.8559 - loss: 0.3970

12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.8555 - loss: 0.3957 - val_accuracy: 0.9318 - val_loss: 0.2630
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 753ms/step - accuracy: 0.8803 - loss: 0.3040

12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.8814 - loss: 0.3024 - val_accuracy: 0.9501 - val_loss: 0.1980
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.9069 - loss: 0.2844

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9084 - loss: 0.2811 - val_accuracy: 0.9606 - val_loss: 0.1624
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 742ms/step - accuracy: 0.9389 - loss: 0.2279

...

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9667 - loss: 0.0961 - val_accuracy: 0.9816 - val_loss: 0.0726
Epoch 16/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 751ms/step - accuracy: 0.9652 - loss: 0.0943

12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9645 - loss: 0.0959 - val_accuracy: 0.9816 - val_loss: 0.0689
Epoch 17/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 762ms/step - accuracy: 0.9652 - loss: 0.1254

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9655 - loss: 0.1244 - val_accuracy: 0.9843 - val_loss: 0.0675
Epoch 18/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 748ms/step - accuracy: 0.9608 - loss: 0.1269

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9610 - loss: 0.1255 - val_accuracy: 0.9869 - val_loss: 0.0672
Epoch 19/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 747ms/step - accuracy: 0.9511 - loss: 0.1028

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9512 - loss: 0.1035 - val_accuracy: 0.9869 - val_loss: 0.0615
Epoch 20/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9556 - loss: 0.1028 - val_accuracy: 0.9869 - val_loss: 0.0631


In [50]:
# --- Save final model ---
model.save("truck_classifier_final_model.h5")

In [51]:
# --- Evaluasi akhir (sementara ke validation dataset) ---
loss, acc = model.evaluate(val_ds)
print(f"\nFinal Validation Accuracy: {acc:.2%}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 576ms/step - accuracy: 0.9910 - loss: 0.0578

Final Validation Accuracy: 98.69%


In [1]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def predict_image(img_path, model_path="truck_classifier_best_model.h5"):
    # Load model
    model = tf.keras.models.load_model(model_path)

    # Load dan proses gambar
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # buat jadi batch [1, 224, 224, 3]
    img_array = img_array / 255.0  # normalisasi

    # Prediksi
    pred = model.predict(img_array)[0][0]
    label = "Truck" if pred >= 0.5 else "Not Truck"
    confidence = pred if pred >= 0.5 else 1 - pred

    # Tampilkan hasil
    plt.imshow(img)
    plt.title(f"{label} ({confidence:.2%})")
    plt.axis('off')
    plt.show()

    print(f"Prediction: {label} | Confidence: {confidence:.2%}")

predict_image("testing/1.jpeg")


NameError: name 'tf' is not defined